In [1]:
pip install streamlit pandas numpy scikit-learn scipy openpyxl plotly

   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.0 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.0 MB 2.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.0 MB 2.2 MB/s eta 0:00:04
   ------ --------------------------------- 1.6/9.0 MB 2.2 MB/s eta 0:00:04
   --------- ------------------------------ 2.1/9.0 MB 2.0 MB/s eta 0:00:04
   ---------- ----------------------------- 2.4/9.0 MB 1.9 MB/s eta 0:00:04
   ---------- ----------------------------- 2.4/9.0 MB 1.9 MB/s eta 0:00:04
   ----------- ---------------------------- 2.6/9.0 MB 1.7 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.0 MB 1.7 MB/s eta 0:00:04
   --------------- ------------------------ 3.4/9.0 MB 1.7 MB/s eta 0:00:04
   ----------------- ---------------------- 3.9/9.0 MB 1.8 MB/s eta 0:00:03
   ------------------- -------------------- 4.5/9.0 MB 1.8 MB/s eta 0:00:03
   -----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
import os
from datetime import datetime

print("✅ All libraries imported successfully")
print(f"📅 Timestamp: {datetime.now()}")

✅ All libraries imported successfully
📅 Timestamp: 2025-12-08 21:43:05.603679


In [3]:
# Path configuration
BASE_PATH = r"C:\Users\muham\Project\Stunting"
DATA_PATH = os.path.join(BASE_PATH, "data_balita.csv")
MODEL_DIR = os.path.join(BASE_PATH, "models")

# Create directories if not exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, "utils"), exist_ok=True)

print(f"📂 Base Path: {BASE_PATH}")
print(f"📂 Data Path: {DATA_PATH}")
print(f"📂 Model Directory: {MODEL_DIR}")

# Check if data file exists
if os.path.exists(DATA_PATH):
    print("✅ Dataset found!")
else:
    print("❌ Dataset not found! Please ensure data_balita.csv is in the correct location.")


📂 Base Path: C:\Users\muham\Project\Stunting
📂 Data Path: C:\Users\muham\Project\Stunting\data_balita.csv
📂 Model Directory: C:\Users\muham\Project\Stunting\models
✅ Dataset found!


In [4]:
# Load dataset
df = pd.read_csv(DATA_PATH)

print("=" * 80)
print("📊 DATASET OVERVIEW")
print("=" * 80)

print(f"\n📏 Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n📋 Columns: {df.columns.tolist()}")

print("\n📊 First 5 rows:")
print(df.head())

print("\n📈 Dataset Info:")
df.info()

print("\n📊 Statistical Summary:")
print(df.describe())

print("\n🎯 Status Gizi Distribution:")
print(df['Status Gizi'].value_counts())
print("\nPercentage:")
print(df['Status Gizi'].value_counts(normalize=True) * 100)

print("\n👥 Gender Distribution:")
print(df['Jenis Kelamin'].value_counts())

print("\n⚠️ Missing Values:")
print(df.isnull().sum())

📊 DATASET OVERVIEW

📏 Dataset Shape: 120999 rows × 4 columns

📋 Columns: ['Umur (bulan)', 'Jenis Kelamin', 'Tinggi Badan (cm)', 'Status Gizi']

📊 First 5 rows:
   Umur (bulan) Jenis Kelamin  Tinggi Badan (cm)       Status Gizi
0             0     laki-laki          44.591973           stunted
1             0     laki-laki          56.705203            tinggi
2             0     laki-laki          46.863358            normal
3             0     laki-laki          47.508026            normal
4             0     laki-laki          42.743494  severely stunted

📈 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120999 entries, 0 to 120998
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Umur (bulan)       120999 non-null  int64  
 1   Jenis Kelamin      120999 non-null  object 
 2   Tinggi Badan (cm)  120999 non-null  float64
 3   Status Gizi        120999 non-null  object 
dtypes: float64(1

In [5]:
print("=" * 80)
print("🔄 DATA PREPROCESSING")
print("=" * 80)

# Remove duplicates
df_clean = df.drop_duplicates()
print(f"\n✅ Removed {df.shape[0] - df_clean.shape[0]} duplicate rows")

# Remove missing values
df_clean = df_clean.dropna()
print(f"✅ Removed rows with missing values")
print(f"📏 Clean dataset shape: {df_clean.shape}")

# Encode categorical features
print("\n🔄 Encoding categorical features...")

# Gender encoding
gender_encoder = LabelEncoder()
df_clean['Jenis Kelamin Encoded'] = gender_encoder.fit_transform(df_clean['Jenis Kelamin'])

# Status Gizi encoding
label_encoder = LabelEncoder()
df_clean['Status Gizi Encoded'] = label_encoder.fit_transform(df_clean['Status Gizi'])

print("\n✅ Encoding completed:")
print("Gender mapping:")
for cls, encoded in zip(gender_encoder.classes_, gender_encoder.transform(gender_encoder.classes_)):
    print(f"  - {cls}: {encoded}")

print("\nStatus Gizi mapping:")
for cls, encoded in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"  - {cls}: {encoded}")

🔄 DATA PREPROCESSING

✅ Removed 81574 duplicate rows
✅ Removed rows with missing values
📏 Clean dataset shape: (39425, 4)

🔄 Encoding categorical features...

✅ Encoding completed:
Gender mapping:
  - laki-laki: 0
  - perempuan: 1

Status Gizi mapping:
  - normal: 0
  - severely stunted: 1
  - stunted: 2
  - tinggi: 3


In [6]:
print("\n" + "=" * 80)
print("🎯 FEATURE ENGINEERING")
print("=" * 80)

# Prepare features (X) and target (y)
feature_columns = ['Umur (bulan)', 'Jenis Kelamin Encoded', 'Tinggi Badan (cm)']
X = df_clean[feature_columns].values
y = df_clean['Status Gizi Encoded'].values

print(f"\n📊 Features shape: {X.shape}")
print(f"🎯 Target shape: {y.shape}")

print(f"\n📋 Feature names: {feature_columns}")
print(f"🎯 Target classes: {label_encoder.classes_.tolist()}")


🎯 FEATURE ENGINEERING

📊 Features shape: (39425, 3)
🎯 Target shape: (39425,)

📋 Feature names: ['Umur (bulan)', 'Jenis Kelamin Encoded', 'Tinggi Badan (cm)']
🎯 Target classes: ['normal', 'severely stunted', 'stunted', 'tinggi']


In [8]:
print("\n" + "=" * 80)
print("✂️ TRAIN-TEST SPLIT")
print("=" * 80)

# Split data dengan stratify untuk menjaga proporsi kelas
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\n📊 Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"📊 Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\n🎯 Training set class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for cls, count in zip(label_encoder.inverse_transform(unique), counts):
    print(f"  - {cls}: {count} ({count/len(y_train)*100:.1f}%)")

print("\n🎯 Test set class distribution:")
unique, counts = np.unique(y_test, return_counts=True)
for cls, count in zip(label_encoder.inverse_transform(unique), counts):
    print(f"  - {cls}: {count} ({count/len(y_test)*100:.1f}%)")


✂️ TRAIN-TEST SPLIT

📊 Training set: 31540 samples (80.0%)
📊 Test set: 7885 samples (20.0%)

🎯 Training set class distribution:
  - normal: 17211 (54.6%)
  - severely stunted: 5216 (16.5%)
  - stunted: 3534 (11.2%)
  - tinggi: 5579 (17.7%)

🎯 Test set class distribution:
  - normal: 4303 (54.6%)
  - severely stunted: 1304 (16.5%)
  - stunted: 883 (11.2%)
  - tinggi: 1395 (17.7%)


In [9]:
print("\n" + "=" * 80)
print("⚖️ FEATURE SCALING")
print("=" * 80)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Features scaled using StandardScaler")
print(f"\n📊 Training set mean: {X_train_scaled.mean(axis=0)}")
print(f"📊 Training set std: {X_train_scaled.std(axis=0)}")


⚖️ FEATURE SCALING

✅ Features scaled using StandardScaler

📊 Training set mean: [-7.38233190e-17  1.37977419e-15 -1.55950219e-14]
📊 Training set std: [1. 1. 1.]


In [10]:
print("\n" + "=" * 80)
print("🔍 HYPERPARAMETER TUNING")
print("=" * 80)

# Define parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

print("\n📋 Parameter grid:")
for param, values in param_grid.items():
    print(f"  - {param}: {values}")

# Grid search
print("\n🔄 Performing Grid Search with 5-fold Cross-Validation...")
knn = KNeighborsClassifier()
grid_search = GridSearchCV(
    knn, 
    param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=2
)

grid_search.fit(X_train_scaled, y_train)

print("\n✅ Grid Search completed!")
print(f"\n🏆 Best parameters: {grid_search.best_params_}")
print(f"🏆 Best cross-validation score: {grid_search.best_score_:.4f}")

# Get best model
best_knn = grid_search.best_estimator_


🔍 HYPERPARAMETER TUNING

📋 Parameter grid:
  - n_neighbors: [3, 5, 7, 9, 11, 13, 15]
  - weights: ['uniform', 'distance']
  - metric: ['euclidean', 'manhattan', 'minkowski']

🔄 Performing Grid Search with 5-fold Cross-Validation...
Fitting 5 folds for each of 42 candidates, totalling 210 fits

✅ Grid Search completed!

🏆 Best parameters: {'metric': 'euclidean', 'n_neighbors': 13, 'weights': 'distance'}
🏆 Best cross-validation score: 0.9887


In [11]:
print("\n" + "=" * 80)
print("📈 MODEL EVALUATION")
print("=" * 80)

# Predictions
y_pred = best_knn.predict(X_test_scaled)
y_pred_train = best_knn.predict(X_train_scaled)

# Calculate metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"\n✅ Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"✅ Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Classification Report
print("\n📊 Classification Report (Test Set):")
print(classification_report(
    y_test, 
    y_pred, 
    target_names=label_encoder.classes_,
    digits=4
))

# Confusion Matrix
print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Detailed confusion matrix with labels
print("\n📊 Detailed Confusion Matrix:")
cm_df = pd.DataFrame(
    cm,
    index=[f"Actual: {cls}" for cls in label_encoder.classes_],
    columns=[f"Predicted: {cls}" for cls in label_encoder.classes_]
)
print(cm_df)

# Cross-validation scores
print("\n🔄 Cross-Validation Scores (5-fold):")
cv_scores = cross_val_score(best_knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Scores: {cv_scores}")
print(f"Mean: {cv_scores.mean():.4f}")
print(f"Std: {cv_scores.std():.4f}")
print(f"95% Confidence Interval: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")



📈 MODEL EVALUATION

✅ Training Accuracy: 1.0000 (100.00%)
✅ Test Accuracy: 0.9920 (99.20%)

📊 Classification Report (Test Set):
                  precision    recall  f1-score   support

          normal     0.9951    0.9956    0.9954      4303
severely stunted     0.9931    0.9893    0.9912      1304
         stunted     0.9729    0.9773    0.9751       883
          tinggi     0.9935    0.9928    0.9932      1395

        accuracy                         0.9920      7885
       macro avg     0.9887    0.9888    0.9887      7885
    weighted avg     0.9920    0.9920    0.9920      7885


📊 Confusion Matrix:
[[4284    0   10    9]
 [   0 1290   14    0]
 [  11    9  863    0]
 [  10    0    0 1385]]

📊 Detailed Confusion Matrix:
                          Predicted: normal  Predicted: severely stunted  \
Actual: normal                         4284                            0   
Actual: severely stunted                  0                         1290   
Actual: stunted                 

In [12]:
print("\n" + "=" * 80)
print("💾 SAVING MODEL & COMPONENTS")
print("=" * 80)

# Save KNN model
model_path = os.path.join(MODEL_DIR, 'knn_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(best_knn, f)
print(f"✅ Model saved: {model_path}")

# Save scaler
scaler_path = os.path.join(MODEL_DIR, 'scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved: {scaler_path}")

# Save encoders
encoders_path = os.path.join(MODEL_DIR, 'encoders.pkl')
with open(encoders_path, 'wb') as f:
    pickle.dump({
        'gender_encoder': gender_encoder,
        'label_encoder': label_encoder
    }, f)
print(f"✅ Encoders saved: {encoders_path}")

# Save model metadata
metadata = {
    'train_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'train_accuracy': float(train_accuracy),
    'test_accuracy': float(test_accuracy),
    'cv_mean': float(cv_scores.mean()),
    'cv_std': float(cv_scores.std()),
    'best_params': grid_search.best_params_,
    'feature_names': feature_columns,
    'target_classes': label_encoder.classes_.tolist(),
    'n_train_samples': int(X_train.shape[0]),
    'n_test_samples': int(X_test.shape[0])
}

metadata_path = os.path.join(MODEL_DIR, 'model_metadata.pkl')
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)
print(f"✅ Metadata saved: {metadata_path}")

print("\n" + "=" * 80)
print("✅ ALL COMPONENTS SAVED SUCCESSFULLY")
print("=" * 80)


💾 SAVING MODEL & COMPONENTS
✅ Model saved: C:\Users\muham\Project\Stunting\models\knn_model.pkl
✅ Scaler saved: C:\Users\muham\Project\Stunting\models\scaler.pkl
✅ Encoders saved: C:\Users\muham\Project\Stunting\models\encoders.pkl
✅ Metadata saved: C:\Users\muham\Project\Stunting\models\model_metadata.pkl

✅ ALL COMPONENTS SAVED SUCCESSFULLY


In [14]:
print("\n" + "=" * 80)
print("🧪 TESTING MODEL PREDICTIONS")
print("=" * 80)

# First, check what gender values are in the dataset
print("\n📋 Gender encoder classes:", gender_encoder.classes_)

# Test cases - updated to match actual dataset gender values
test_cases = [
    {'age': 12, 'gender': gender_encoder.classes_[0], 'height': 70, 'expected': 'Possibly stunted'},
    {'age': 24, 'gender': gender_encoder.classes_[0], 'height': 87, 'expected': 'Normal'},
    {'age': 36, 'gender': gender_encoder.classes_[1], 'height': 85, 'expected': 'Possibly stunted'},
    {'age': 48, 'gender': gender_encoder.classes_[0], 'height': 103, 'expected': 'Normal'},
    {'age': 6, 'gender': gender_encoder.classes_[1], 'height': 60, 'expected': 'Possibly stunted'}
]

print("\n📝 Running test predictions...\n")

for i, case in enumerate(test_cases, 1):
    # Encode gender
    gender_encoded = gender_encoder.transform([case['gender']])[0]
    
    # Prepare input
    X_test_case = np.array([[case['age'], gender_encoded, case['height']]])
    X_test_case_scaled = scaler.transform(X_test_case)
    
    # Predict
    pred_encoded = best_knn.predict(X_test_case_scaled)[0]
    prediction = label_encoder.inverse_transform([pred_encoded])[0]
    
    # Get probabilities
    probabilities = best_knn.predict_proba(X_test_case_scaled)[0]
    prob_dict = dict(zip(label_encoder.classes_, probabilities))
    
    # Calculate stunting risk
    risk_classes = ['severely stunted', 'stunted']
    risk_percentage = sum([prob_dict.get(cls, 0) for cls in risk_classes]) * 100
    
    print(f"Test Case {i}:")
    print(f"  Input: Age={case['age']} months, Gender={case['gender']}, Height={case['height']}cm")
    print(f"  Prediction: {prediction}")
    print(f"  Risk: {risk_percentage:.2f}%")
    print(f"  Probabilities:")
    for status, prob in prob_dict.items():
        print(f"    - {status}: {prob*100:.2f}%")
    print()


🧪 TESTING MODEL PREDICTIONS

📋 Gender encoder classes: ['laki-laki' 'perempuan']

📝 Running test predictions...

Test Case 1:
  Input: Age=12 months, Gender=laki-laki, Height=70cm
  Prediction: stunted
  Risk: 100.00%
  Probabilities:
    - normal: 0.00%
    - severely stunted: 0.00%
    - stunted: 100.00%
    - tinggi: 0.00%

Test Case 2:
  Input: Age=24 months, Gender=laki-laki, Height=87cm
  Prediction: normal
  Risk: 0.00%
  Probabilities:
    - normal: 100.00%
    - severely stunted: 0.00%
    - stunted: 0.00%
    - tinggi: 0.00%

Test Case 3:
  Input: Age=36 months, Gender=perempuan, Height=85cm
  Prediction: stunted
  Risk: 100.00%
  Probabilities:
    - normal: 0.00%
    - severely stunted: 0.00%
    - stunted: 100.00%
    - tinggi: 0.00%

Test Case 4:
  Input: Age=48 months, Gender=laki-laki, Height=103cm
  Prediction: normal
  Risk: 0.00%
  Probabilities:
    - normal: 100.00%
    - severely stunted: 0.00%
    - stunted: 0.00%
    - tinggi: 0.00%

Test Case 5:
  Input: Age=6

In [15]:
print("\n" + "=" * 80)
print("📋 MODEL TRAINING SUMMARY")
print("=" * 80)

print(f"""
✅ Model Training Completed Successfully!

📊 Dataset Information:
   - Total samples: {len(df_clean)}
   - Training samples: {len(X_train)}
   - Test samples: {len(X_test)}
   - Features: {len(feature_columns)}
   - Classes: {len(label_encoder.classes_)}

🎯 Model Performance:
   - Training Accuracy: {train_accuracy*100:.2f}%
   - Test Accuracy: {test_accuracy*100:.2f}%
   - CV Mean Score: {cv_scores.mean()*100:.2f}%
   - CV Std: {cv_scores.std()*100:.2f}%

🏆 Best Hyperparameters:
   - n_neighbors: {grid_search.best_params_['n_neighbors']}
   - weights: {grid_search.best_params_['weights']}
   - metric: {grid_search.best_params_['metric']}

💾 Saved Files:
   - {os.path.basename(model_path)}
   - {os.path.basename(scaler_path)}
   - {os.path.basename(encoders_path)}
   - {os.path.basename(metadata_path)}

📁 Location: {MODEL_DIR}

🚀 Next Steps:
   1. Model sudah siap digunakan
   2. Jalankan Streamlit app dengan command:
      streamlit run app.py
   3. Buka browser di localhost:8501
""")

print("=" * 80)


📋 MODEL TRAINING SUMMARY

✅ Model Training Completed Successfully!

📊 Dataset Information:
   - Total samples: 39425
   - Training samples: 31540
   - Test samples: 7885
   - Features: 3
   - Classes: 4

🎯 Model Performance:
   - Training Accuracy: 100.00%
   - Test Accuracy: 99.20%
   - CV Mean Score: 98.87%
   - CV Std: 0.05%

🏆 Best Hyperparameters:
   - n_neighbors: 13
   - weights: distance
   - metric: euclidean

💾 Saved Files:
   - knn_model.pkl
   - scaler.pkl
   - encoders.pkl
   - model_metadata.pkl

📁 Location: C:\Users\muham\Project\Stunting\models

🚀 Next Steps:
   1. Model sudah siap digunakan
   2. Jalankan Streamlit app dengan command:
      streamlit run app.py
   3. Buka browser di localhost:8501



In [ ]:
print("\n🚀 Launching Streamlit App...")
print("="*80)

# Pastikan file app.py ada di directory yang sama

import subprocess
import sys

# Get the directory of this notebook
notebook_dir = BASE_PATH

print("\n📝 To run the Streamlit app manually, execute in terminal:")
print(f"   cd {BASE_PATH}")
print(f"   {sys.executable} -m streamlit run app.py")
print("\nOr simply:")
print("   python -m streamlit run app.py")

print("\n" + "=" * 80)
print("✅ NOTEBOOK EXECUTION COMPLETED")
print("=" * 80)
print("\n💡 Note: If 'streamlit' command not found, use:")
print("   python -m streamlit run app.py")
print("\n🌐 The app will open in your browser at: http://localhost:8501")


🚀 Launching Streamlit App...
